In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
import re
import string
import sklearn
from sklearn_pandas import DataFrameMapper
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from skopt import BayesSearchCV
from tqdm import tqdm_notebook as tqdm
from reed import *
from cinspect import dependence, importance
from sklearn.model_selection import cross_val_score, cross_validate
import pickle
import time



# set global notebook options
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
%matplotlib inline

%load_ext autoreload
%autoreload 2

sklearn.__version__

## Response Model

How well can we predict outcomes $Y$ conditional on treatment $T$ and other covariates $Z$?
   - fit ML models on kitchen sink, Anna's set & basic set
   - fit basic LR on basic set

### Treatent variables


   - **redhllt**, 
   - **redllt** 
   - **refllt** 
   - **reduhl**	Completed re-education based on highest level of attainment
   - **redudl**	Completed re-education based on detailed qualifications
   - **redufl**	Completed re-education using highest lvl and detailed qualifications.

### Outcome variables
   - Mental health in 2019 (**mh**). This is the transformed mental health scores from the aggregation of mental health items of the SF-36 Health Survey, as reported by the individual in 2019. It ranges from 0 to 100, with higher scores indicating better mental health.  
   - Working hours in 2019 (**wkhr**) records the total number of hours the individual works in all jobs in a week on average. Working hours are set to 0 for those not working. 
   - Hourly Wages in 2019 (**rlwage**) records the average hourly wage for the individual’s main job in 2019. Hourly wages are set to 0 for those not working and set to missing for those reporting working more than 100 hours a week. 

#### Columns explicitly excluded
   - **xwaveid** (unique identifier)
   - **p_rcom*** (timing of completion of re-education, proxies treatment) TODO think about how we would include this
   - **p_cotrl** (first avail 2003)
   - **p_rdf*** (first avail 2012)

In [ ]:
treatments = ['^reduhl$', '^rehllt$', '^redudl$', '^redufl$', '^redllt$', '^refllt$']
outcomes = ['^rlwage$', '^mh$', '^mhbm$', '^wkhr$']
other = [
            '^p_rcom',
            '^p_rdf',
            '^p_cotrl',
            '^xwaveid$',
            'p_rcom18'  # ?
            '^aedcq',  # indicate studying at start - these people should already have been removed
            '^abnfsty',
            '^aedcqfpt',
            '^aedqstdy'
]
exclude = treatments + outcomes + other


outcome = 'rlwage'
treatment = 'redudl'
optimisation_metric = 'neg_mean_squared_error'

## Data

In [ ]:
meta, basic, df, raw = load_all_data()
for d in [basic, df, raw]:
    drop_missing_treatment_or_outcome(d, treatment, outcome)

### Set up models

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

def construct_models():
    models = [
        Model('ridge',Ridge(), 
              parameters = {
                  'alpha':np.logspace(-1,4,30)
              }
        ),
        Model('lasso',Lasso(),
              parameters = {
                  'alpha':np.logspace(-2,4,30)
              }
        ), 
        Model('gbr',GradientBoostingRegressor(n_iter_no_change=20, max_depth=2),
              parameters = {
                'max_features':[10,20,40,60,80],
                'learning_rate':np.logspace(-3,0,10),
                'min_samples_leaf':np.logspace(0,3,10).astype(int)
              }
        ),
    ]
    return models

### Fit models and visualise performance

In [ ]:
evaluation_metrics = metrics = ['r2','neg_mean_squared_error']

transform = Pipeline([
    ('impute_missing', SimpleImputer()),
    ('scale', StandardScaler()),
])

data = raw

control, treated = treatment_control_split(data, treatment)
features = regex_select(data.columns, exclude, exclude=True)
X0,y0 = split_and_transform(control, features, outcome, transform)
X1,y1 = split_and_transform(treated, features, outcome, transform)

In [ ]:
# construct the full dataset (remove ordering by treatment in case of any order dependance in fit)
X = np.vstack((X0,X1))
y = np.concatenate((y0,y1))
indx = np.arange(len(y))
np.random.shuffle(indx)
X = X[indx,:]
y = y[indx]

### (Nested) cross-validate to evaluate model performance

In [ ]:
def nested_cross_val(load_from_cache=False, cache_name = "nested_cv_results.pkl"):
    if load_from_cache:
        with open(cache_name,'rb') as f:
            models, results = pickle.load(f)
            
    else:
        models = construct_models()
        results = {}
        for model in models:
            print(f"Fitting {model.name} ...",end='')
            results0 = model.nested_cv_fit_evaluate(X0,y0,optimisation_metric,evaluation_metrics)
            results1 = model.nested_cv_fit_evaluate(X1,y1,optimisation_metric,evaluation_metrics)
            results[model.name] = (results0, results1)
            print("Done")
        
        print(f"Caching results to {cache_name}")
        with open(cache_name,'wb') as f:
            pickle.dump((models,results),f)
            
    return models,results
 

models, results = nested_cross_val(load_from_cache=True)

#### Visualise and report model performance

  - Mean and Std of prediction performance for each model (both treatment & control surface)
  - Mean and Std of average treatment effect for each model
  - Features responsible for treatment effect heterogeneity & functional form (with uncertainty)
      - coefficeints for linear models
      - partial dependence curves for non-linear models

In [ ]:
invalid = 1000 # threshold to avoid displaying results that failed to converge entirely

rows = []
index = []
for model_name, r in results.items():
    tau = estimate_causal_effect(X, r[0]['estimator'],r[1]['estimator'])
    row = {'ACE':tau.mean(),'ACE_std':tau.std()}
    for m in evaluation_metrics:
        key = f'test_{m}'
        for name, result in zip(('control','treated'),r):
            label=f"{name}_{m}"
            label_std=f"{label}_std"
            row[label]= result[key].mean()
            row[label_std] = result[key].std()
    rows.append(row)
    index.append(model_name)
metrics = pd.DataFrame(rows,index=index)
metrics[metrics.abs()> invalid] = np.nan

with pd.option_context('display.float_format', '{:,.2f}'.format):
    display(metrics)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,5),sharey=True)
ax[0].bar(metrics.index, metrics['control_r2'], yerr=metrics['control_r2_std'], align='center', alpha=0.5, capsize=10)
ax[1].bar(metrics.index, metrics['treated_r2'], yerr=metrics['treated_r2_std'], align='center', alpha=0.5,capsize=10)
ax[0].set_ylabel('$R^2$')
ax[0].set_title('control model')
ax[1].set_title('treated model');

### Bootstraped cross-validation to estimate parameter uncertainty

In [ ]:
def extract_params(estimator):
    return estimator.coef_

def bootstrapped_cross_val(load_from_cache=False, cache_name="bootstrap_cv_results.pkl", samples=10):
    if load_from_cache:
        with open(cache_name, 'rb') as f:
            results = pickle.load(f)
    else:
        models = construct_models()
        results = {}
        start = time.time()
        for model in models:
            print(f"Fitting {model.name} ...",end='')
            results0 = model.bootstrap_cv_evaluate(X0,y0,optimisation_metric,extract_params,
                                                   bootstrap_samples=samples,return_estimator=True)
            results1 = model.bootstrap_cv_evaluate(X1,y1,optimisation_metric,extract_params,
                                                   bootstrap_samples=samples,return_estimator=True)
            results[model.name] = (results0, results1)
            print("Done")
        total = time.time()-start
        print(f"Total time:{total} seconds")
        print(f"Caching results to: {cache_name}")
        with open(cache_name,'wb') as f:
            pickle.dump(results,f)
    
    return results

bootstrap_results = bootstrapped_cross_val(load_from_cache=False,samples=100)

### Bootstrapped uncertainty on average treatment effects

In [ ]:
def estimate_causal_effect(X, models0, models1):
    tau = []
    for e0, e1 in zip(models0,models1):
        y0 = e0.predict(X)
        y1 = e1.predict(X)
        tau.append(y1-y0)
    
    # array of shape len(modelsi),len(X)
    cate = np.array(tau) 
    
    # array of shape len(modelsi) with the ate estimate for each sample
    ate = np.mean(cate,axis=1) 
    return ate

In [ ]:
for model_name, (results0, results1) in bootstrap_results.items():
    models0 = [r['estimator'] for r in results0]
    models1 = [r['estimator'] for r in results1]
    ate = estimate_causal_effect(X,models0, models1)
    print(model_name, ate.mean(),ate.std()/np.sqrt(len(ate)-1))  

### Visualise distribution of hyper-parameters

In [ ]:
from collections import defaultdict
def hyperparam_distributions(samples) -> {str:[]}:
    """Returns a dict from hyper-parameter name to the best values for that hyper-parameter over the samples."""
    distributions = defaultdict(list)
    for sample in samples:
        h = sample['estimator'].best_params_
        for key, value in h.items():
            distributions[key].append(value)
    return distributions

def plot_hyperparam_distributions(samples, title) -> None:
    distributions = hyperparam_distributions(samples)
    k = len(distributions)
    fig, axes = plt.subplots(1,k,figsize=(k*5,4))
    if k == 1:
        axes = [axes]
    for i, (key, values) in enumerate(distributions.items()):
        ax = axes[i]
        ax.hist(values)
        ax.set_title(title)
        ax.set_xlabel(key)
        ax.set_ylabel('count')
    return fig,axes

for model, (results0, results1) in bootstrap_results.items():
    plot_hyperparam_distributions(results0,f"{model}-control")
    plot_hyperparam_distributions(results1,f"{model}-treated")

In [ ]:
# TODO, could also think about visualising distribution of coefficeints for linear models. 
# TODO, why is this results so different to what I am getting from T-learners in econml